Implementation of [Frame Interpolation for Large Scene Motion code repo](https://github.com/google-research/frame-interpolation). All credit goes to them. 

[Original notebook](https://colab.research.google.com/drive/1tbbbnQge0yb0LmnWNchEKNhjtBNC6jX-#scrollTo=UAuGknLNtVQF) maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616

Ported to paperspace by [@proximasan](https://twitter.com/proximasan)

Release v1.2
- fixed crash that prevented interpolation. BIG BIG shoutout to [Alec](https://twitter.com/ai_for_humans) for debugging and discovering the root cause!

Release v1.3
- Raise user warning for a space in frames_folder (instead of horribly crashing in an obscure way)
- Small UX improvement for relative and absolute frames path

In [3]:
# @title Set up the project

from pathlib import Path
import os

ROOT_FOLDER = "generative" # @param {type:"string"}
PROJECT_FOLDER = "frame-interpolation" # @param {type:"string"}
BASE_PATH = Path('/notebooks')
PROJECT_PATH = BASE_PATH / ROOT_FOLDER / PROJECT_FOLDER

os.makedirs(PROJECT_PATH, exist_ok=True)

In [4]:
# @title Setup
!git clone https://github.com/google-research/frame-interpolation frame_interpolation
!pip install mediapy==1.0.3 gdown tensorflow==2.8.0 natsort
!pip install tqdm

fatal: destination path 'frame_interpolation' already exists and is not an empty directory.


In [5]:
# @title Imports

from pathlib import Path
import functools
import os
from typing import List, Sequence
import sys
import glob
import math
from base64 import b64encode

from IPython import display
from frame_interpolation.eval import interpolator as interpolator_lib
from frame_interpolation.eval import util
from absl import app
from absl import flags
from absl import logging
import mediapy as media
import natsort
import numpy as np
import tensorflow as tf
from tqdm.notebook import trange


## Setup Models

In [ ]:
# Download Models from Huggingface 

import requests
import zipfile
import io

# Store the URL in a variable
url = "https://huggingface.co/proxima/FILM_pretrained_models/resolve/main/pretrained_models.zip"

# Set the pretrained models path
pretrained_models_path = "/notebooks/generative/frame-interpolation/pretrained_models"

# Download the zip file
response = requests.get(url)

# Check if the download was successful (status code 200)
if response.status_code == 200:
    # Open the downloaded zip file as a byte stream
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Extract the zip file to the specified directory
    zip_file.extractall(pretrained_models_path)
    print("Downloaded and extracted the zip file successfully.")
else:
    print("Error: Could not download the zip file. Please check the URL.")

## Interpolate

Frames png files should be in the frames folder in alphabetical order frame.png files. Files will be processed in alphabetical order
- `01_frame.png` 
- `02_frame.png` 
- `03_frame.png`

times_to_interpolate:
- The number of times to run recursive midpoint interpolation. The number of output frames will be 2^times_to_interpolate+1. Longer is slower and smoother but takes longer to process

transition_pause, first_frame_pause, video_fps;
- Relative to video FPS. If FPS is 30, 0.3 will pause for 9 framess. Set to 0 to turn off

video_style:
- loop: Create a continuous loop with the first frame. Create a smooth loop back to the start
- foward_reverse: forward until the final frame. Then reverse back to the start. Warning: Doubles processing time.
- normal: Start at frame 1 and progress to the final frame


flim_net_model:
- Style: crisp but can produce hard edges and jitters
- VGG: trade off between crisp and smooth
- L1: smoother but blurrier

In [8]:
#@markdown Frames png files should be in the frames folder in alphabetical order frame.png files. Files will be processed in alphabetical order
#@markdown - `01_frame.png` 
#@markdown - `02_frame.png` 
#@markdown - `03_frame.png`
#@markdown 
#@markdown Use a relative path in the project folder i.e. `my_folder`. Or an absolute path such as `/content/my_folder`
frames_folder = "/notebooks/yourframes" #@param{type:"string"}
#@markdown The number of times to run recursive midpoint interpolation. The number of output frames will be 2^times_to_interpolate+1. Longer is slower and smoother but takes longer to process.
times_to_interpolate = 7 #@param{type:"slider", min:0, max:10, step:1}
#@markdown Relative to video FPS. If FPS is 30, 0.3 will pause for 9 framess. Set to 0 to turn off.
transition_pause =  0.3#@param{type:"number"}
# pad start
first_frame_pause =  0.1#@param{type:"number"}
# default 30. 'Frames per second to play interpolated videos in slow motion.'
video_fps = 30 #@param{type:"number"}
# Style of the video output
#@markdown - `loop`: Create a continuous loop with the first frame. Create a smooth loop back to the start
#@markdown - `foward_reverse`: forward until the final frame. Then reverse back to the start. Warning: Doubles processing time.
#@markdown - `normal`: Start at frame 1 and progress to the final frame
video_style = "normal" #@param ['loop', 'foward_reverse', 'normal']
# select model
#@markdown - `Style`:  crisp but can produce hard edges and jitters
#@markdown - `VGG`: trade off between crisp and smooth
#@markdown - `L1`: smoother but blurrier
flim_net_model = "VGG" #@param ["VGG", "Style", "L1"]

# Add other extensions, if not either.
INPUT_EXT = ['.png', '.jpg', '.jpeg']

# a space in frames folder causes havoc with ffmpeg
if " " in frames_folder:
    msg = f"""

Frames_folder cannot cantain a space in the name. 
Please replace the space with _ or -
    """
    raise UserWarning(msg)


# intput/output directors
frames_folder = Path(frames_folder)
if frames_folder.is_absolute():
    frames_path = frames_folder
else:
    frames_path = PROJECT_PATH / frames_folder

intermediate_output = frames_path / 'intermediate_videos'

# collect input frames
input_files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{str(frames_path)}/*"))) 
               if f.is_file() and f.suffix in INPUT_EXT]

if not input_files:
    msg = f"""

No images found in folder: {frames_path}
This folder does not exist or contains no image files.
    """
    raise UserWarning(msg)

print("Interpolating", len(input_files), "images")

videos_list_file = f"{frames_path}/videos_files_list.txt"

# clear old frames
!rm "$videos_list_file"
!rm -r "$intermediate_output"
!mkdir -p "$intermediate_output"

ffmpeg_path = util.get_ffmpeg_path()
media.set_ffmpeg(ffmpeg_path)

if video_style == "loop":
    input_files.append(input_files[0])
elif video_style == "foward_reverse":
    input_files += input_files[1::-1]

frame_sets = list(zip(input_files[:-1], input_files[1:]))

# calculate padding
t_padding_frames = math.floor(video_fps * transition_pause)
s_padding_frames = math.floor(video_fps * first_frame_pause)


# load the model
print("Loading the model...")
model_path = str(PROJECT_PATH / f"pretrained_models/pretrained_models/film_net/{flim_net_model}/saved_model")
interpolator = interpolator_lib.Interpolator(model_path)

# interpolate
for i in trange(len(frame_sets)):
    infiles = frame_sets[i]
    frames = list(util.interpolate_recursively_from_files(infiles, times_to_interpolate, interpolator))

    # pad the start of the animation if on the first frame
    if i == 0:
        frames = [frames[0]] * s_padding_frames + frames
 
    frames += [frames[-1]] * t_padding_frames

    # output_frames(frames, str(frames_output_path))
    output_mp4 = f'{intermediate_output}/interpolated_{i:04}.mp4'
    media.write_video(output_mp4, frames, fps=video_fps)

file_namespace = os.path.basename(os.path.normpath(frames_folder)).replace(" ", "_")
final_filename = f"{file_namespace}_{video_fps}FPS_{video_style}.mp4"
print(f'Saving final video to {final_filename}')
all_videos = [f"file '{f}'\n"
              for f in map(Path, sorted(glob.glob(f"{str(intermediate_output)}/*"))) 
              if f.is_file() and f.suffix == ".mp4"]

with open(videos_list_file, "w") as f:
    f.writelines(all_videos)

final_output_path = f"{frames_path}/{final_filename}"
!ffmpeg -y -f concat -safe 0 -i $videos_list_file -c copy "$final_output_path"

Interpolating 2 images
Loading the model...


  0%|          | 0/1 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████| 127/127 [00:16<00:00,  7.92it/s]


Saving final video to yourframes_30FPS_normal.mp4
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-lib

In [9]:
#@title Play Video

mp4 = open(final_output_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
